In [1]:
import time
import random
from pathlib import Path

import cv2
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

from pycoral.utils.edgetpu import make_interpreter
from pycoral.adapters import classify
from pycoral.adapters import common

import tflite_runtime.interpreter as tflite
from imgaug import augmenters as img_aug

from lane_navigation.image_augmentation import random_augment


In [2]:
def my_imread(image_path):
    image = cv2.imread(str(image_path))
    # image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2YUV)
    return image


def img_preprocess(image):
    height, _, _ = image.shape
    image = image[
        int(height / 2) :, :, :
    ]  # remove top half of the image, as it is not relavant for lane following
    image = cv2.cvtColor(
        image, cv2.COLOR_RGB2YUV
    )  # Nvidia model said it is best to use YUV color space
    image = cv2.GaussianBlur(image, (3, 3), 0)
    image = cv2.resize(image, (200, 66))  # input image size (200,66) Nvidia model
    # image = image / 255 # normalizing, the processed image becomes black for some reason.  do we need this?
    image = image.astype(np.uint8)
    return image


def image_data_generator(image_paths, batch_size, is_training):
    while True:
        batch_images = []
        batch_steering_angles = []

        for i in range(batch_size):
            random_index = random.randint(0, len(image_paths) - 1)
            image_path = image_paths[random_index]
            image = my_imread(image_paths[random_index])
            if is_training:
                # training: augment image
                image, steering_angle = random_augment(image, steering_angle)

            image = img_preprocess(image)
            batch_images.append(image)

        yield np.asarray(batch_images)


def set_input_tensor(interpreter, input):
    input_details = interpreter.get_input_details()[0]
    tensor_index = input_details["index"]
    input_tensor = interpreter.tensor(tensor_index)()
    # Inputs for the TFLite model must be uint8, so we quantize our input data.
    scale, zero_point = input_details["quantization"]
    quantized_input = np.uint8(input / scale + zero_point)
    input_tensor[:, :, :] = quantized_input


def predict_steer(interpreter, input):
    set_input_tensor(interpreter, input)
    interpreter.invoke()
    output_details = interpreter.get_output_details()[0]
    output = interpreter.get_tensor(output_details["index"])
    # Outputs from the TFLite model are uint8, so we dequantize the results:
    scale, zero_point = output_details["quantization"]
    output = scale * (output - zero_point)
    return output


In [3]:
image_paths = []
lab_dirs = list(Path("train_data_generation/data/drive_with_keypress/").glob("*"))
for lab_dir in lab_dirs:
    frame_paths = list(lab_dir.glob("*.png"))
    for frame_path in frame_paths:
        image_paths.append(frame_path)
image_paths.sort()


In [4]:
lane_nav_model_path = Path("lane_navigation/model/lane_navigation_final.h5")
lane_nav_model = tf.keras.models.load_model(lane_nav_model_path)


2022-10-23 14:08:04.849736: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-23 14:08:04.856698: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-23 14:08:04.857165: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-23 14:08:04.857907: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

In [5]:
def representative_data_gen():
    batch_size = 4
    input_data = tf.cast(
        next(image_data_generator(image_paths, batch_size, False)), dtype=tf.float32
    )
    yield [input_data]


converter = tf.lite.TFLiteConverter.from_keras_model(lane_nav_model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_data_gen
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.target_spec.supported_types = [tf.int8]
converter.inference_input_type = tf.uint8
converter.inference_output_type = tf.uint8

lane_nav_tflite_model = converter.convert()

with open("lane_navigation/model/lane_navigation_final.tflite", "wb") as f:
    f.write(lane_nav_tflite_model)


2022-10-23 14:08:05.704290: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: /tmp/tmpinoa7ymq/assets


2022-10-23 14:08:06.512302: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:363] Ignored output_format.
2022-10-23 14:08:06.512324: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:366] Ignored drop_control_dependency.
2022-10-23 14:08:06.512821: I tensorflow/cc/saved_model/reader.cc:43] Reading SavedModel from: /tmp/tmpinoa7ymq
2022-10-23 14:08:06.514687: I tensorflow/cc/saved_model/reader.cc:107] Reading meta graph with tags { serve }
2022-10-23 14:08:06.514699: I tensorflow/cc/saved_model/reader.cc:148] Reading SavedModel debug info (if present) from: /tmp/tmpinoa7ymq
2022-10-23 14:08:06.521186: I tensorflow/cc/saved_model/loader.cc:210] Restoring SavedModel bundle.
2022-10-23 14:08:06.591021: I tensorflow/cc/saved_model/loader.cc:194] Running initialization op on SavedModel bundle at path: /tmp/tmpinoa7ymq
2022-10-23 14:08:06.606195: I tensorflow/cc/saved_model/loader.cc:283] SavedModel load for tags { serve }; Status: success: OK. Took 93376 

In [6]:
# image = cv2.imread(
#     "train_data_generation/data/drive_with_keypress/5/frame_000016_102.png"
# )
# image = cv2.imread(
#     "train_data_generation/data/drive_with_keypress/8/frame_000027_60.png"
# )
# image = cv2.imread(
#     "train_data_generation/data/drive_with_keypress/9/frame_000001_91.png"
# )
image = cv2.imread(
    "train_data_generation/data/drive_with_keypress/28/frame_000017_90.png"
)

image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

proc_image = np.asarray([img_preprocess(image)])

plt.imshow(proc_image[0])


In [7]:
lane_nav_model.predict(proc_image)[0]

2022-10-23 14:08:21.907555: I tensorflow/stream_executor/cuda/cuda_dnn.cc:366] Loaded cuDNN version 8101
2022-10-23 14:08:22.293343: W tensorflow/stream_executor/gpu/asm_compiler.cc:111] *** WARNING *** You are using ptxas 10.1.243, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.

You may not need to update to CUDA 11.1; cherry-picking the ptxas binary is often sufficient.
2022-10-23 14:08:22.294257: W tensorflow/stream_executor/gpu/asm_compiler.cc:230] Falling back to the CUDA driver for PTX compilation; ptxas does not support CC 8.6
2022-10-23 14:08:22.294264: W tensorflow/stream_executor/gpu/asm_compiler.cc:233] Used ptxas at ptxas
2022-10-23 14:08:22.294313: W tensorflow/stream_executor/gpu/redzone_allocator.cc:314] UNIMPLEMENTED: ptxas ptxas too old. Falling back to the driver to compile.
Relying on driver to perform ptx compilation. 
Modify $PATH to customize ptxas location.
This message will be 

array([91.68618], dtype=float32)

In [8]:
interpreter = tflite.Interpreter(
    model_path="lane_navigation/model/lane_navigation_final.tflite"
)
interpreter.allocate_tensors()

predict_steer(interpreter, proc_image)


array([[92.08881569]])

In [10]:
interpreter = make_interpreter(
    "lane_navigation/model/lane_navigation_final_edgetpu.tflite"
)
interpreter.allocate_tensors()

predict_steer(interpreter, proc_image)


array([[92.08881569]])